<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


### 0. Imports

In [1]:
import pandas as pd

import sys
sys.path.append("..")

from src.support_database_connect import establecer_conn, crear_db
import src.support_database_connect as sdc

### 1. Data import

In [2]:
demanda_evolucion_df = pd.read_csv("../../Lab 2/data/processed/demanda_evolucion.csv",index_col=0)
generacion_renovables_df = pd.read_csv("../../Lab 2/data/processed/generacion_renovables.csv",index_col=0)
demograficos_INE_df = pd.read_csv("../../Lab 2/data/processed/demographic_INE.csv",index_col=0)
economicos_INE_df = pd.read_csv("../../Lab 2/data/processed/economic_INE.csv",index_col=0)

### 2. Data re-exploration

In [3]:
demanda_evolucion_df.head()

,value,datetime,ccaa,cod_ccaa,outliers
0,17830.239,2018-12-31 23:00:00,Ceuta,8744,0.0
1,16019.527,2019-01-31 23:00:00,Ceuta,8744,0.0
2,16598.235,2019-02-28 23:00:00,Ceuta,8744,0.0
3,15221.094,2019-03-31 22:00:00,Ceuta,8744,0.0
4,16753.540,2019-04-30 22:00:00,Ceuta,8744,0.0


In [4]:
generacion_renovables_df.head()

,value,percentage,datetime,type,ccaa,cod_ccaa,type_ccaa_outliers
0,5.053,0.009113,2018-12-31 23:00:00,Solar fotovoltaica,Melilla,8745,0
1,5.526,0.011159,2019-01-31 23:00:00,Solar fotovoltaica,Melilla,8745,0
2,6.561,0.012039,2019-02-28 23:00:00,Solar fotovoltaica,Melilla,8745,0
3,7.925,0.016925,2019-03-31 22:00:00,Solar fotovoltaica,Melilla,8745,0
4,8.683,0.016244,2019-04-30 22:00:00,Solar fotovoltaica,Melilla,8745,0


In [5]:
demograficos_INE_df.head(5)

,provinces,age,nationality,gender,year,total,total.1,cod_province
0,Albacete,0-15 años,Españoles,Hombres,2021,27.304,27304,2
1,Albacete,0-15 años,Españoles,Hombres,2020,27.979,27979,2
2,Albacete,0-15 años,Españoles,Hombres,2019,28.513,28513,2
3,Albacete,0-15 años,Españoles,Mujeres,2021,25.547,25547,2
4,Albacete,0-15 años,Españoles,Mujeres,2020,25.953,25953,2


In [6]:
economicos_INE_df.head()

,provinces,activity,period,total,cod_province
0,Albacete,producto interior bruto a precios de mercado,2021,8663814,2
1,Albacete,producto interior bruto a precios de mercado,2020,7855556,2
2,Albacete,producto interior bruto a precios de mercado,2019,8475070,2
3,Alicante/Alacant,producto interior bruto a precios de mercado,2021,37279735,3
4,Alicante/Alacant,producto interior bruto a precios de mercado,2020,33979911,3


INE data is given by provinces and year, while the REE data has the data by region and month. That makes it tricky to join both unless having them both aggregated to the year and CCAA level. For that reason, the data models will not really be connected in the database model.

### 3. Decision on columns to keep or transform

#### 3.1 Economicos INE

In [7]:
economicos_INE_df.astype("O").describe()

,provinces,activity,period,total,cod_province
count,156,156,156,156,156
unique,52,1,3,156,52
top,Albacete,producto interior bruto a precios de mercado,2021,8663814,2
freq,3,156,52,1,3


From economicos_INE_df:
- activity: drop. It has an only unique value.
- provinces: drop. Leave cod_province as id_province to connect it with table 'provinces'

New table - 'provinces':
- id_province (cod_province)
- province_name

In [8]:
provinces = economicos_INE_df[["provinces","cod_province"]].drop_duplicates()
economicos_INE_df.drop(columns=["provinces","activity"],inplace=True)

#### 3.2 Demograficos INE

In [9]:
demograficos_INE_df.astype("O").describe()

,provinces,age,nationality,gender,year,total,total.1,cod_province
count,1872,1872,1872,1872,1872,1872,1872,1872
unique,52,3,2,2,3,1820,1820,52
top,Albacete,0-15 años,Españoles,Hombres,2021,410,410,2
freq,36,624,936,936,624,3,3,36


From demograficos_INE_df:
- provinces: drop. Leave cod_province as id_province to connect it with table 'provinces'

In [10]:
demograficos_INE_df.drop(columns=["provinces"],inplace=True)

#### 3.3 Generacion renovables

In [11]:
generacion_renovables_df.astype("O").describe()

,value,percentage,datetime,type,ccaa,cod_ccaa,type_ccaa_outliers
count,2909.000,2909.0,2909,2909,2909,2909,2909
unique,2909.000,2907.0,36,7,18,18,2
top,700.953,0.0,2019-03-31 22:00:00,Solar fotovoltaica,Cataluña,9,0
freq,1.000,3.0,81,647,216,216,2829


From generacion_renovables_df:
- type_ccaa_outliers: drop. It can be calculated anytime.
- percentage: drop. It can be calculated any time.
- type: could be replaced by an id, but there are no other characteristics to add and, given the size of the data space saving is negligible.
- ccaa: drop. Leave cod_ccaa and create new ccaa column.
- datetime: dive into year and month

New table - ccaa
- ccaa_id
- ccaa

In [12]:
generacion_renovables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2909 entries, 0 to 2908
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value               2909 non-null   float64
 1   percentage          2909 non-null   float64
 2   datetime            2909 non-null   object 
 3   type                2909 non-null   object 
 4   ccaa                2909 non-null   object 
 5   cod_ccaa            2909 non-null   int64  
 6   type_ccaa_outliers  2909 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 181.8+ KB


In [13]:
generacion_renovables_df["year"] = pd.to_datetime(generacion_renovables_df["datetime"]).dt.year
generacion_renovables_df["month"] = pd.to_datetime(generacion_renovables_df["datetime"]).dt.month
ccaa = generacion_renovables_df[["ccaa","cod_ccaa"]].drop_duplicates()

In [14]:
generacion_renovables_df.drop(columns=["type_ccaa_outliers","ccaa","datetime","percentage"])

,value,type,cod_ccaa,year,month
0,5.053,Solar fotovoltaica,8745,2018,12
1,5.526,Solar fotovoltaica,8745,2019,1
2,6.561,Solar fotovoltaica,8745,2019,2
3,7.925,Solar fotovoltaica,8745,2019,3
4,8.683,Solar fotovoltaica,8745,2019,4
...,...,...,...,...,...
2904,645.334,Otras renovables,20,2021,7
2905,695.538,Otras renovables,20,2021,8
2906,588.318,Otras renovables,20,2021,9
2907,434.937,Otras renovables,20,2021,10


#### 3. 4 Demanda evolucion

In [15]:
demanda_evolucion_df.astype("O").describe()

,value,datetime,ccaa,cod_ccaa,outliers
count,684.000,684,684,684,684.0
unique,684.000,36,19,19,2.0
top,141258.077,2018-12-31 23:00:00,Ceuta,8744,0.0
freq,1.000,19,36,36,653.0


From demanda_evolucion_df:
- outliers: drop. It can be calculated anytime.
- type: could be replaced by an id, but there are no other characteristics to add and, given the size of the data so space saving is negligible.
- ccaa: drop. Leave cod_ccaa and create new ccaa column.
- datetime: dive into year and month

In [16]:
demanda_evolucion_df["year"] = pd.to_datetime(demanda_evolucion_df["datetime"]).dt.year
demanda_evolucion_df["month"] = pd.to_datetime(demanda_evolucion_df["datetime"]).dt.month

### Proposed database schema

```sql
CREATE TABLE ccaa (
    ccaa_id INT PRIMARY KEY,
    ccaa VARCHAR(100) NOT NULL
);

CREATE TABLE provinces (
    province_id INT PRIMARY KEY,
    province_name VARCHAR(100) NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);

CREATE TABLE economicos (
    economicos_id SERIAL PRIMARY KEY,
    province_id INT REFERENCES provinces(province_id) ON DELETE SET NULL,
    year INT NOT NULL,
    total DECIMAL(15, 2) NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);

CREATE TABLE demograficos (
    demograficos_id SERIAL PRIMARY KEY,
    year INT NOT NULL,
    province_id INT REFERENCES provinces(province_id) ON DELETE SET NULL,
    age_range VARCHAR(20d) NOT NULL,
    nationality VARCHAR(100) NOT NULL,
    gender CHAR(1) CHECK (gender IN ('H', 'M')),
    total INT NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);

CREATE TABLE generacion_renovables (
    generacion_id SERIAL PRIMARY KEY,
    year INT NOT NULL,
    month INT CHECK (month BETWEEN 1 AND 12),
    value DECIMAL(15, 2) NOT NULL,
    type VARCHAR(100) NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);

CREATE TABLE demanda_evolucion (
    demanda_id SERIAL PRIMARY KEY,
    year INT NOT NULL,
    month INT CHECK (month BETWEEN 1 AND 12),
    value DECIMAL(15, 2) NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);
```

In [17]:
crear_db("lab3_mod5")

La base de datos ya existe


In [18]:
conn = establecer_conn("lab3_mod5", "admin", "postgres")

conn.cursor().execute(sdc.drop_all_tables)
conn.cursor().execute(sdc.ccaa_table)
conn.cursor().execute(sdc.provinces_table)
conn.cursor().execute(sdc.economicos_table)
conn.cursor().execute(sdc.demograficos_table)
conn.cursor().execute(sdc.demanda_evolucion_table)
conn.cursor().execute(sdc.generacion_renovables_table)

### 4. Insert tables

In [19]:
cur = conn.cursor()

#### 4.1 Insert ccaa records

In [20]:
ccaa_tuples = [tuple(row) for _,row in ccaa.iterrows()]
ccaa_tuples

[('Melilla', 8745),
 ('Andalucía', 4),
 ('Aragón', 5),
 ('Cantabria', 6),
 ('Castilla - La Mancha', 7),
 ('Castilla y León', 8),
 ('Cataluña', 9),
 ('País Vasco', 10),
 ('Principado de Asturias', 11),
 ('Comunidad de Madrid', 13),
 ('Comunidad Foral de Navarra', 14),
 ('Comunitat Valenciana', 15),
 ('Extremadura', 16),
 ('Galicia', 17),
 ('Illes Balears', 8743),
 ('Canarias', 8742),
 ('Región de Murcia', 21),
 ('La Rioja', 20)]

In [21]:
cur.executemany(
"INSERT INTO ccaa (ccaa,ccaa_id) VALUES (%s,%s)",
ccaa_tuples
)

#### 4.2 Insert provinces 

In [22]:
province_to_ccaa = {
    "Albacete": 7,
    "Alicante/Alacant": 15,
    "Almería": 4,
    "Araba/Álava": 10,
    "Asturias": 11,
    "Ávila": 8,
    "Badajoz": 16,
    "Balears, Illes": 8743,
    "Barcelona": 9,
    "Bizkaia": 10,
    "Burgos": 8,
    "Cáceres": 16,
    "Cádiz": 4,
    "Cantabria": 6,
    "Castellón/Castelló": 15,
    "Ciudad Real": 7,
    "Córdoba": 4,
    "Coruña, A": 17,
    "Cuenca": 7,
    "Gipuzkoa": 10,
    "Girona": 9,
    "Granada": 4,
    "Guadalajara": 7,
    "Huelva": 4,
    "Huesca": 5,
    "Jaén": 4,
    "León": 8,
    "Lleida": 9,
    "Lugo": 17,
    "Madrid": 13,
    "Málaga": 4,
    "Murcia": 21,
    "Navarra": 14,
    "Ourense": 17,
    "Palencia": 8,
    "Palmas, Las": 8742,
    "Pontevedra": 17,
    "Rioja, La": 20,
    "Salamanca": 8,
    "Santa Cruz de Tenerife": 8742,
    "Segovia": 8,
    "Sevilla": 4,
    "Soria": 8,
    "Tarragona": 9,
    "Teruel": 5,
    "Toledo": 7,
    "Valencia/València": 15,
    "Valladolid": 8,
    "Zamora": 8,
    "Zaragoza": 5,
    "Ceuta": 8745,
    "Melilla": 8745
}


Adding the ccaa_id

In [23]:
provinces["ccaa_id"] =  provinces["provinces"].map(lambda x: province_to_ccaa.get(x))
provinces = provinces[[ "cod_province", "provinces","ccaa_id"]]

In [24]:
provinces_tuples = [tuple(row) for _,row in provinces.iterrows()]
provinces_tuples[:5]

[(2, 'Albacete', 7),
 (3, 'Alicante/Alacant', 15),
 (4, 'Almería', 4),
 (1, 'Araba/Álava', 10),
 (33, 'Asturias', 11)]

In [25]:
cur.executemany(
"INSERT INTO provinces VALUES (%s,%s,%s)",
provinces_tuples
)

#### 4.3 Insert economicos

In [26]:
province_id_to_ccaa_id_mapping = {int(row[0]):int(row[1]) for _,row in provinces[["cod_province","ccaa_id"]].iterrows()}
economicos_INE_df["ccaa_id"] =  economicos_INE_df["cod_province"].map(lambda x: province_id_to_ccaa_id_mapping.get(x))

C:\Users\Miguel\AppData\Local\Temp\ipykernel_22456\2191284678.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  province_id_to_ccaa_id_mapping = {int(row[0]):int(row[1]) for _,row in provinces[["cod_province","ccaa_id"]].iterrows()}


In [27]:
economicos_INE_df = economicos_INE_df[["cod_province","period","total","ccaa_id"]]

In [28]:
economicos_INE_tuples = [tuple(row) for _,row in economicos_INE_df.iterrows()]
economicos_INE_tuples[:5]

[(2, 2021, 8663814, 7),
 (2, 2020, 7855556, 7),
 (2, 2019, 8475070, 7),
 (3, 2021, 37279735, 15),
 (3, 2020, 33979911, 15)]

In [29]:
cur.executemany(
"INSERT INTO economicos (province_id, year, total, ccaa_id) VALUES (%s,%s,%s,%s)",
economicos_INE_tuples
)

#### 4.4 Insert demográficos

In [30]:
demograficos_INE_df["gender"] = demograficos_INE_df["gender"].str[:1]

In [31]:
demograficos_INE_df = demograficos_INE_df[["year","cod_province","age","nationality","gender","total.1"]]

In [32]:
demograficos_INE_df["ccaa_id"] =  demograficos_INE_df["cod_province"].map(lambda x: province_id_to_ccaa_id_mapping.get(x))

In [33]:
demograficos_INE_tuple = [tuple(row) for _,row in demograficos_INE_df.iterrows()]
demograficos_INE_tuple[:5]

[(2021, 2, '0-15 años', 'Españoles', 'H', 27304, 7),
 (2020, 2, '0-15 años', 'Españoles', 'H', 27979, 7),
 (2019, 2, '0-15 años', 'Españoles', 'H', 28513, 7),
 (2021, 2, '0-15 años', 'Españoles', 'M', 25547, 7),
 (2020, 2, '0-15 años', 'Españoles', 'M', 25953, 7)]

In [34]:
cur.executemany(
"INSERT INTO demograficos (year, province_id, age, nationality, gender, total, ccaa_id) VALUES (%s,%s,%s,%s,%s,%s,%s)",
demograficos_INE_tuple
)

#### 4.5 Insert generacion renovables

In [35]:
generacion_renovables_df = generacion_renovables_df[["year","month","value","type","cod_ccaa"]]
generacion_renovables_tuple = [tuple(row) for _,row in generacion_renovables_df.iterrows()]
generacion_renovables_tuple[:5]

[(2018, 12, 5.053, 'Solar fotovoltaica', 8745),
 (2019, 1, 5.526, 'Solar fotovoltaica', 8745),
 (2019, 2, 6.561, 'Solar fotovoltaica', 8745),
 (2019, 3, 7.925, 'Solar fotovoltaica', 8745),
 (2019, 4, 8.683, 'Solar fotovoltaica', 8745)]

In [36]:
cur.executemany(
"INSERT INTO generacion_renovables (year, month, value, type, ccaa_id) VALUES (%s,%s,%s,%s,%s)",
generacion_renovables_tuple
)

#### 4.6 Insert demanda evolucion    

CREATE TABLE demanda_evolucion (
    demanda_id SERIAL PRIMARY KEY,
    year INT NOT NULL,
    month INT CHECK (month BETWEEN 1 AND 12),
    value DECIMAL(15, 2) NOT NULL,
    ccaa_id INT REFERENCES ccaa(ccaa_id) ON DELETE SET NULL
);

In [37]:
demanda_evolucion_df

,value,datetime,ccaa,cod_ccaa,outliers,year,month
0,17830.239,2018-12-31 23:00:00,Ceuta,8744,0.0,2018,12
1,16019.527,2019-01-31 23:00:00,Ceuta,8744,0.0,2019,1
2,16598.235,2019-02-28 23:00:00,Ceuta,8744,0.0,2019,2
3,15221.094,2019-03-31 22:00:00,Ceuta,8744,0.0,2019,3
4,16753.540,2019-04-30 22:00:00,Ceuta,8744,0.0,2019,4
...,...,...,...,...,...,...,...
679,130850.035,2021-07-31 22:00:00,La Rioja,20,0.0,2021,7
680,133323.629,2021-08-31 22:00:00,La Rioja,20,0.0,2021,8
681,135918.940,2021-09-30 22:00:00,La Rioja,20,0.0,2021,9
682,140846.250,2021-10-31 23:00:00,La Rioja,20,0.0,2021,10


In [38]:
demanda_evolucion_df = demanda_evolucion_df[["year","month","value","cod_ccaa"]]

In [39]:
demanda_evolucion_tuple = [tuple(row) for _,row in demanda_evolucion_df.iterrows()]
demanda_evolucion_tuple[:5]

[(2018.0, 12.0, 17830.239, 8744.0),
 (2019.0, 1.0, 16019.527, 8744.0),
 (2019.0, 2.0, 16598.235, 8744.0),
 (2019.0, 3.0, 15221.094, 8744.0),
 (2019.0, 4.0, 16753.54, 8744.0)]

In [40]:
ccaa

,ccaa,cod_ccaa
0,Melilla,8745
24,Andalucía,4
84,Aragón,5
132,Cantabria,6
192,Castilla - La Mancha,7
252,Castilla y León,8
300,Cataluña,9
372,País Vasco,10
432,Principado de Asturias,11
480,Comunidad de Madrid,13


In [ ]:
cur.executemany(
"INSERT INTO demanda_evolucion (year, month, value, ccaa_id) VALUES (%s,%s,%s,%s)",
demanda_evolucion_tuple
)

ForeignKeyViolation: inserción o actualización en la tabla «demanda_evolucion» viola la llave foránea «demanda_evolucion_ccaa_id_fkey»
DETAIL:  La llave (ccaa_id)=(8744) no está presente en la tabla «ccaa».


Creating the ccaa tables, value for Ceuta was not added:

In [45]:
cur.execute(
"INSERT INTO ccaa (ccaa,ccaa_id) VALUES (%s,%s)",
("Ceuta",8744)
)

It should stop raising the ForeignKeyViolation now:

In [46]:
cur.executemany(
"INSERT INTO demanda_evolucion (year, month, value, ccaa_id) VALUES (%s,%s,%s,%s)",
demanda_evolucion_tuple
)